In [1]:
import pandas as pd

# Load the datasets with only the required columns
df1 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/final_animedataset.csv', usecols=['user_id', 'anime_id', 'my_score', 'score', 'scored_by', 'gender', 'popularity'])
df2 = pd.read_csv('C:/Users/Guest01/Documents/dataset_anime/archive/users-score-2023.csv', usecols=['user_id', 'anime_id', 'Anime Title', 'rating'])

# Merge the datasets on user_id and anime_id
final_df = pd.merge(df2, df1, on=['user_id', 'anime_id'])

# Display the first few rows to check the data
print(final_df.head())


   user_id  anime_id             Anime Title  rating  my_score gender  score  \
0        1        21               One Piece       9         9   Male   8.54   
1        1        48             .hack//Sign       7         7   Male   7.09   
2        1       320                  A Kite       5         5   Male   6.66   
3        1        49        Aa! Megami-sama!       8         8   Male   7.38   
4        1       304  Aa! Megami-sama! Movie       8         8   Male   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


In [2]:
# Frequency encoding for categorical columns
categorical_columns = ['gender', 'Anime Title']

for col in categorical_columns:
    freq = final_df[col].value_counts()
    final_df[col] = final_df[col].map(freq)

# Handle missing values by filling with median for numerical columns
numerical_columns = ['my_score', 'score', 'scored_by', 'popularity']
for col in numerical_columns:
    final_df[col].fillna(final_df[col].median(), inplace=True)

# Display the first few rows after preprocessing
print(final_df.head())


C:\Users\Guest01\AppData\Local\Temp\ipykernel_19152\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_19152\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

   user_id  anime_id  Anime Title  rating  my_score   gender  score  \
0        1        21        10462       9         9  4175028   8.54   
1        1        48         5341       7         7  4175028   7.09   
2        1       320         1892       5         5  4175028   6.66   
3        1        49         2448       8         8  4175028   7.38   
4        1       304         2395       8         8  4175028   7.63   

   scored_by  popularity  
0     423868          35  
1      61485         650  
2      18934        1946  
3      20930        1807  
4      18571        2007  


C:\Users\Guest01\AppData\Local\Temp\ipykernel_19152\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df[col].fillna(final_df[col].median(), inplace=True)
C:\Users\Guest01\AppData\Local\Temp\ipykernel_19152\4112127917.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

In [3]:
# Define features (X) and target (y)
X = final_df.drop(columns=['rating'])  # All columns except 'rating' are features
y = final_df['rating']  # 'rating' is the target variable

# Display the shapes of X and y to confirm
print(f'X shape: {X.shape}, y shape: {y.shape}')


X shape: (6021419, 8), y shape: (6021419,)


In [4]:
from sklearn.model_selection import train_test_split

# Function to sample data
def sample_data(X, y, sample_size):
    if isinstance(sample_size, float):
        if 0 < sample_size < 1.0:
            return train_test_split(X, y, test_size=0.2, train_size=sample_size, random_state=42)
        elif sample_size == 1.0:
            return train_test_split(X, y, test_size=0.001, random_state=42)
        else:
            raise ValueError("sample_size as float must be in the range (0.0, 1.0) or equal to 1.0.")
    elif isinstance(sample_size, int):
        if sample_size > len(X):
            raise ValueError(f"sample_size {sample_size} exceeds the number of available samples {len(X)}.")
        sampled_X = X.sample(n=sample_size, random_state=42)
        sampled_y = y.loc[sampled_X.index]
        return train_test_split(sampled_X, sampled_y, test_size=0.2, random_state=42)
    else:
        raise ValueError("sample_size must be a float or an integer")


In [5]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import time
import psutil
import numpy as np

# Function to calculate and return metrics
def calculate_metrics(X_train, X_test, y_train, y_test):
    xgb = XGBRegressor(tree_method = 'hist', device='cuda')

    # Define hyperparameters for RandomizedSearchCV
    param_distributions = {
        'n_estimators': [30, 50],  # Increase range for more boosting stages
        'learning_rate': [0.05, 0.1],  # Add a higher learning rate
        'max_depth': [3, 5, 7, 10],  # Increase depth for more complex models -> tweak this
        'min_child_weight': [1, 2, 5],  # Increase the range for more control over fitting
        'subsample': [0.8, 1.0],  # Add more options for the subsample parameter
        'colsample_bytree': [0.8, 1.0],  # Add more options for column sampling
    }

    random_search = RandomizedSearchCV(xgb, param_distributions, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42, n_jobs=-1)

    start_time = time.time()
    start_cpu = psutil.cpu_percent(interval=None)
    random_search.fit(X_train, y_train)
    end_cpu = psutil.cpu_percent(interval=None)
    end_time = time.time()

    # Calculate time and CPU usage
    execution_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    y_pred = random_search.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    # Calculate the range of the target variable
    target_range = y_train.max() - y_train.min()

    # Calculate normalized RMSE (nRMSE)
    nrmse = rmse / target_range
    
    memory_usage_MB = X_train.memory_usage(deep=True).sum() / (1024 ** 2)
    normalized_time = execution_time / memory_usage_MB
    
    return {
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'MSE': mse,
        'nRMSE': nrmse,  # Normalized RMSE
        'Execution Time (Raw)': execution_time,  # Raw execution time
        'Normalized Time (s/MB)': normalized_time,  # Normalized execution time
        'Average CPU Usage': avg_cpu_usage
    }


In [6]:
# Define sample sizes
sample_sizes = [1.0, 0.5, 0.25, 0.125, 100, 1000, 10000, 100000]

# Initialize metrics storage
metrics_list = []
total_execution_time = 0
total_cpu_usage = 0
total_memory_usage_MB = 0

# Loop through each sample size
for size in sample_sizes:
    try:
        X_train_sample, X_test_sample, y_train_sample, y_test_sample = sample_data(X, y, size)
        metrics = calculate_metrics(X_train_sample, X_test_sample, y_train_sample, y_test_sample)
        metrics['Sample Size'] = size
        metrics_list.append(metrics)

        # Accumulate total       
        total_execution_time += metrics['Execution Time (Raw)']
        total_cpu_usage += metrics['Average CPU Usage']
        total_memory_usage_MB += X_train_sample.memory_usage(deep=True).sum() / (1024 ** 2)

        print(f"Metrics for sample size {size}:")
        for key, value in metrics.items():
            print(f"{key}: {value}")
        print("-" * 50)

    except Exception as e:
        print(f"An error occurred for sample size {size}: {e}")


c:\Users\Guest01\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [09:05:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Metrics for sample size 1.0:
RMSE: 0.26046793200699386
MAPE: 0.016143773042398973
R2: 0.9755885004997253
MSE: 0.06784354360399997
nRMSE: 0.02894088133411043
Execution Time (Raw): 379.64621210098267
Normalized Time (s/MB): 0.9191411142753985
Average CPU Usage: 59.85
Sample Size: 1.0
--------------------------------------------------
Metrics for sample size 0.5:
RMSE: 0.2771125159296063
MAPE: 0.017170487067350783
R2: 0.9723504185676575
MSE: 0.0767913464848363
nRMSE: 0.030790279547734032
Execution Time (Raw): 65.60822343826294
Normalized Time (s/MB): 0.3173634539055234
Average CPU Usage: 58.15
Sample Size: 0.5
--------------------------------------------------
Metrics for sample size 0.25:
RMSE: 0.27719768054971006
MAPE: 0.017186015019646766
R2: 0.9723334312438965
MSE: 0.0768385541021391
nRMSE: 0.030799742283301117
Execution Time (Raw): 35.677730083465576
Normalized Time (s/MB): 0.34516439599367765
Average CPU Usage: 68.0
Sample Size: 0.25
-------------------------------------------------

In [7]:
# Convert metrics to DataFrame
metrics_df = pd.DataFrame(metrics_list)

# Calculate total metrics
total_avg_cpu_usage = total_cpu_usage / len(sample_sizes)
normalized_total_time = total_execution_time / total_memory_usage_MB

# Convert total execution time to minutes and seconds
total_minutes = int(total_execution_time // 60)
total_seconds = total_execution_time % 60

# Display total metrics
print(f"Total Execution Time for Entire Process (Raw): {total_minutes} minutes and {total_seconds:.2f} seconds")
print(f"Total Normalized Execution Time for Entire Process: {normalized_total_time:.8f} seconds per MB")
print(f"Total Average CPU Usage for Entire Process: {total_avg_cpu_usage:.2f}%")

# Display the metrics DataFrame
metrics_df


Total Execution Time for Entire Process (Raw): 8 minutes and 54.05 seconds
Total Normalized Execution Time for Entire Process: 0.68386769 seconds per MB
Total Average CPU Usage for Entire Process: 69.66%


,RMSE,MAPE,R2,MSE,nRMSE,Execution Time (Raw),Normalized Time (s/MB),Average CPU Usage,Sample Size
0,0.260468,0.016144,0.975589,0.067844,0.028941,379.646212,0.919141,59.85,1.000
1,0.277113,0.017170,0.972350,0.076791,0.030790,65.608223,0.317363,58.15,0.500
2,0.277198,0.017186,0.972333,0.076839,0.030800,35.677730,0.345164,68.00,0.250
3,0.277593,0.017208,0.972254,0.077058,0.030844,23.909894,0.462633,73.60,0.125
4,0.692372,0.095704,0.890051,0.479380,0.086547,4.916784,895.073134,81.30,100.000
5,0.181530,0.015054,0.987904,0.032953,0.020170,5.802664,105.634275,72.45,1000.000
6,0.329839,0.018235,0.963310,0.108793,0.036649,7.762677,14.131523,73.65,10000.000
7,0.271478,0.017026,0.973576,0.073700,0.030164,10.723891,1.952225,70.25,100000.000
